In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd
import openai

### Read the sampled dataset with Amazon inventory data

In [2]:
df_items = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [3]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[],169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...",[],24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN


In [4]:
list(df_items["features"].items())[0]

(0,
 ['【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger)',
  '【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 10 lite and more',
  '【Premium Workmanship】Unique increased friction design allows you to easily unplug the cable from your charger,combine 250d bulletproof fiber core to build a cable so durable

In [5]:
list(df_items["images"].items())[0]

(0,
 [{'thumb': 'https://m.media-amazon.com/images/I/51G07yWoOBL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51G07yWoOBL.jpg',
   'variant': 'MAIN',
   'hi_res': 'https://m.media-amazon.com/images/I/611AVJMH+JL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41c+40oKkKL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41c+40oKkKL.jpg',
   'variant': 'PT01',
   'hi_res': 'https://m.media-amazon.com/images/I/61ihhPW7VCL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/51y1YnwiUZL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51y1YnwiUZL.jpg',
   'variant': 'PT02',
   'hi_res': 'https://m.media-amazon.com/images/I/61UkcVETKcL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41Nvr++q39L._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41Nvr++q39L.jpg',
   'variant': 'PT03',
   'hi_res': 'https://m.media-amazon.c

### Preprocess title and features

In [6]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [7]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

In [8]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_large_image, axis=1)

In [9]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,image
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51G07yWoOB...
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41bOA5-ogW...
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,USB C Docking Station Dual Monitor for MacBook...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN,https://m.media-amazon.com/images/I/416IzmVKiC...
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[2023 Upgraded] Telescopes for Adults Astronom...,169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41wO4J3TT0...
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...","Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41mwlYqT5p...


In [10]:
list(df_items["description"].items())[0]

(0,
 "RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB Type C Fast Charging Cord - Nylon Braided USB C Charger Cable for Galaxy A20/A50/S10/S9/S8+/S8, iPad Pro 2018, Sony XZ, HTC 10, OnePlus 5T, Huawei P9 etc. 【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger) 【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 1

### Sample 50 items from the dataset

In [11]:
df_sample = df_items.sample(50, random_state=42)

In [12]:
len(df_sample)

50

In [13]:
data_to_embed = df_sample[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [14]:
data_to_embed

[{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- 

### Define the embedding function

In [15]:
response = openai.embeddings.create(
    input="Random text",
    model="text-embedding-3-small",
)

In [16]:
len(response.data[0].embedding)

1536

In [17]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

In [18]:
get_embedding("Hi")

[-0.0069594960659742355,
 -0.035274259746074677,
 0.0015957315918058157,
 0.06534460932016373,
 0.03293841332197189,
 -0.024201158434152603,
 -0.02610827423632145,
 0.04937804862856865,
 0.01623266376554966,
 -0.05168433114886284,
 -0.013357206247746944,
 -0.014599049463868141,
 -0.026019571349024773,
 -0.003257990349084139,
 0.024585537612438202,
 0.001171619864180684,
 -0.05345839262008667,
 0.015057348646223545,
 0.011487049050629139,
 0.03394371271133423,
 0.04934848099946976,
 0.020372141152620316,
 -0.01396334357559681,
 0.01887897402048111,
 0.017149262130260468,
 0.024156806990504265,
 0.01827283576130867,
 -0.0011956436792388558,
 0.01955902948975563,
 -0.03678221255540848,
 0.027675362303853035,
 -0.028207581490278244,
 0.027645794674754143,
 -0.01623266376554966,
 -0.011716199107468128,
 -0.01604047417640686,
 -0.01407422311604023,
 0.03758053854107857,
 0.01887897402048111,
 -0.037698812782764435,
 0.04343494400382042,
 -0.012411040253937244,
 0.020948711782693863,
 0.01355

### Create Qdrant collection

In [19]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [20]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

### Embed data

##### Test

In [21]:
pointstruct = PointStruct(
    id=0,
    vector=get_embedding("Test text"),
    payload={
        "text": "Test text",
        "model": "text-embedding-3-small",
    },
)

In [22]:
pointstruct

PointStruct(id=0, vector=[-0.020074957981705666, 0.00697109242901206, 0.03770574927330017, -0.0402989499270916, -0.01915094256401062, -0.03430775925517082, 0.000514635001309216, -0.024233022704720497, 0.03871918469667435, 0.0016561467200517654, 0.03061170130968094, 0.012772263027727604, -0.010849717073142529, 0.011267013847827911, 0.02639402262866497, 0.04304118454456329, -0.04685647040605545, -0.006859316490590572, -0.021744145080447197, 0.05105924606323242, 0.007105223368853331, 0.013107591308653355, 0.011326627805829048, -0.03251934424042702, -0.003908431623131037, -0.039047058671712875, -0.03693076968193054, 0.005029916763305664, 0.05848116800189018, -0.07463651895523071, 0.03147610276937485, -0.04408442601561546, -0.0002682622289285064, -0.010991299524903297, 0.006054529454559088, 0.03821246698498726, 0.030313633382320404, 0.03761632740497589, 0.010290837846696377, -0.029359811916947365, -0.014888553880155087, -0.010469678789377213, 0.021222524344921112, 0.01819712109863758, 0.026

### Amazon data

In [23]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embediing = get_embedding(data["description"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embediing,
            payload=data,
        )
    )

In [24]:
pointstructs

[PointStruct(id=0, vector=[0.010930042713880539, -0.015056160278618336, 0.01137808058410883, -0.011284304782748222, -0.0503886416554451, -0.0004936232580803335, -0.03859378397464752, 0.04171963036060333, 0.002547564683482051, -0.010346551425755024, -0.0291745662689209, 0.014462249353528023, -0.03096671774983406, 0.08677349239587784, 0.01424343977123499, 0.004819013178348541, -0.03596807271242142, 0.009127471596002579, -0.018807174637913704, 0.016889989376068115, 0.01028924435377121, 0.0174943208694458, 0.033800818026065826, 0.03059161640703678, 0.021776730194687843, 0.02110988274216652, -0.03313397243618965, -0.026548855006694794, -0.023027068004012108, 0.05222247168421745, -0.013222330249845982, -0.020338840782642365, -0.01941150613129139, -0.049179982393980026, -0.04113613814115524, -0.019265633076429367, -0.017202574759721756, 0.01579594425857067, -0.018254943192005157, 0.0005453950725495815, 0.024214889854192734, 0.04019838199019432, 0.00795527920126915, 0.0011038144584745169, -0.0

In [25]:
len(pointstructs)

50

### Write embedded data to Qdrant

In [26]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs,
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Define a function for data retrieval

In [27]:
def retrieve_data(query, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    return results

### Test retrieval

In [28]:
retrieve_data("What kind of charging cords do you offer?", k=10).points

[ScoredPoint(id=4, version=1, score=0.4796399, payload={'description': 'iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] Apple MFi Certified USB A Charging Cable for iPhone 13 12 11 Mini Pro XR Xs Max X SE 8 7 6 Plus iPad iPod AirPods - Black Original Apple Cable - iPhone charging cable with a reinforced joint design that has passed 30,000+ times bending tests for extra protection and durability. Insulation and precisely layer-welded connectors, which make the cable more durable and sturdier than normal iPhone charger cables but also flexible and tangle-free. Instant Responses - No error message pops up. Real high-speed iPhone charging cable with stable and efficient current output, you can plug it into laptop computers and in-car charging adapter. Super Fast Charge - The iPhone Lightning cable build-in four-core high-quality copper wires and multiple safety protections overcharge/stable current/automatic switching/battery protection to promote maximum signal qu